In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [49]:
import sys
import preprocess

In [51]:
print(dir(preprocess))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'get_data', 'np', 'reduce', 'tf']


In [46]:
file_path = "../data/clean_data.csv"
data = pd.read_csv(file_path)
print(data.shape)
data.head()

(2000, 5)


,About the game,Reviews,Header image,Screenshots,Genres
0,In this action-loaded arcade game you play the...,"“In the end it is a game with a unusual name, ...",https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,"Action,Casual,Indie,Racing,Simulation"
1,Rogue Islands is an action-FPS set in a whimsi...,“The sleeper hit of 2017” Trusted Reviews “It’...,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,"Action,Adventure,Indie,RPG,Strategy"
2,How long can your family survive during the su...,"“Foxfolk is a very simple game, but features s...",https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,"Action,Adventure,Casual,Indie"
3,A surreal 3D platformer about a marble girl na...,“Clever design and strange imagery meld togeth...,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,"Action,Indie"
4,Praetorians is set amidst the political machin...,“Praetorians is an excellent choice for real-t...,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,"Indie,Strategy"


In [42]:
import random
def split_train_test(data):
    size = data.shape[0]
    train_size = int(size * 0.8)
    train_indices = random.sample(range(size), train_size)
    train_data = data.iloc[train_indices]

    all_indices = np.arange(data.shape[0])
    test_indices = np.setdiff1d(all_indices, train_indices)
    test_data = data.iloc[test_indices]

    train_data.to_csv('train.csv', index=True)
    test_data.to_csv('test.csv', index=True)

    print('<Done splitting train and test!>')

    return train_data, test_data

In [44]:
_ = split_train_test(data)

<Done splitting train and test!>


In [19]:
train_file = '../data/nlp_train.txt'
test_file = '../data/nlp_test.txt'
vocabulary, vocab_size, train_data, test_data = {}, 0, [], []

with open(train_file, 'r', encoding='utf-8') as file:
    for line in file:
        sentence_list = line.lower().split()
        for word in sentence_list:
            train_data.append(word)

# built vocab list
train_unique_words = sorted(set(train_data))
vocabulary = {w:i for i, w in enumerate(train_unique_words)}
# vocabulary['<UNK>'] = len(vocabulary)

with open(test_file, 'r', encoding='utf-8') as file:
        for line in file:
            sentence_list = line.lower().split()
            for word in sentence_list:
                if word in vocabulary:  
                    test_data.append(word)
                else:
                    test_data.append(vocabulary['<UNK>'])

In [20]:
test_data

['rogue',
 'islands',
 'is',
 'an',
 31485,
 'set',
 'in',
 'a',
 'whimsical',
 'fantasy',
 'world',
 'with',
 'procedural',
 'levels',
 'and',
 'permadeath.',
 'you',
 'control',
 'a',
 'nimble,',
 31485,
 31485,
 'wizard',
 'named',
 31485,
 'he',
 'can',
 'run,',
 'jump,',
 31485,
 'swim',
 'and',
 'cast',
 'powerful',
 'magical',
 'spells.',
 'what',
 'you',
 'lack',
 'in',
 'armor',
 'and',
 'defenses,',
 'you',
 'make',
 'up',
 'for',
 'in',
 'speed',
 'and',
 'magic.',
 'enemies',
 31485,
 'walk,',
 'fly',
 'and',
 'swim',
 'across',
 'the',
 'world',
 'with',
 31485,
 'they',
 'are',
 'powerful,',
 'numerous',
 'and',
 31485,
 'there',
 'are',
 '7',
 'unique',
 'island',
 'types,',
 'brimming',
 'with',
 'forests,',
 'mountains,',
 'caves,',
 'ruins,',
 'creatures',
 'and',
 'bosses.',
 'you',
 'control',
 'a',
 'nimble,',
 31485,
 31485,
 'wizard',
 'named',
 31485,
 'he',
 'can',
 'run,',
 'jump,',
 31485,
 'swim',
 'and',
 'cast',
 'powerful',
 'magical',
 'spells.',
 'the',

In [ ]:
%%time

import rnn

train_id, test_id, vocab = preprocess.get_data("../data/train.txt", "../data/test.txt")

train_id = np.array(train_id)
test_id  = np.array(test_id)
X0, Y0 = train_id[:-1], train_id[1:]
X1, Y1  = test_id[:-1],  test_id[1:]
print(X0.shape, Y0.shape)

## TODO: Reshape the inputs and outputs (X0, Y0, X1, Y1) into the RNN shape
def process_data(window_size, data):
    remainder = (len(data) - 1)%window_size
    data = data[:-remainder]
    data = data[:-1].reshape(-1, 20)
    return data

X0 = process_data(20, X0)
Y0 = process_data(20, Y0)
X1 = process_data(20, X1)
Y1 = process_data(20, Y1)
print(X0.shape, Y0.shape)
